In [1]:
import mysql.connector
import pandas as pd 
import json
import numpy as np
import MySQLdb
import MySQLdb.cursors

In [2]:
# daya pembeda = (2(KA - KB))/n
# daya pembeda = pA - pB

# bawah = nilai rendah
# atas = nilai tinggi

In [14]:
def init_soal(paket_id):
    sql = "SELECT * FROM soal where paket_id='%s'" % (paket_id)
    soal = pd.read_sql(sql, con=db_thread)
    soal = soal.set_index('id')
    soal['benar_bawah'] = 0
    soal['benar_atas'] = 0
    return soal

In [15]:
def hitung_benar(ujian_siswas, keterangan):
    for ujian_siswa in ujian_siswas:
        soals = json.loads(ujian_siswa["random_soal"])
        for soal_id in soals:
            benar_siswa = json.loads(soal.loc[soal_id, 'siswa_id_benar'])
            if(soal.loc[soal_id, 'tipe_soal'] == 'pilihan_ganda'):
                if(ujian_siswa['id'] in benar_siswa): 
                    soal.loc[soal_id, keterangan] += 1

In [16]:
def get_ujian_siswa_kategori(kategori, paket_id, batas_jumlah):
    sql = "select * from ujian_siswa u WHERE u.paket_id='%s' AND u.status=3 ORDER BY u.jumlah_benar %s LIMIT %d" %(paket_id, kategori, batas_jumlah)
    cur_thread.execute(sql)
    ujian_siswas = cur_thread.fetchall()
    return ujian_siswas

In [19]:
db_thread = MySQLdb.connect("127.0.0.1", "root", "andhika", "usek_monev", cursorclass=MySQLdb.cursors.DictCursor)
cur_thread = db_thread.cursor()
print(cur_thread)

cur_thread.execute("SELECT * FROM paket u WHERE status=3")
pakets = cur_thread.fetchall()
for paket in pakets:
    soal = init_soal(paket['id'])
    
    sql = "select count(*) as jumlah from ujian_siswa where paket_id='%s' and status=3" % (paket['id'])
    cur_thread.execute(sql)
    ujian_siswas = cur_thread.fetchall()
    
    if ujian_siswas[0]['jumlah'] > 0:
        if(ujian_siswas[0]['jumlah'] > 200):
            batas_jumlah = int(ujian_siswas[0]['jumlah']*0.27)
        else:
            batas_jumlah = int(ujian_siswas[0]['jumlah']/2)

        #nilai tinggi
        ujian_siswas_atas = get_ujian_siswa_kategori('DESC', paket['id'], batas_jumlah)
        keterangan = 'benar_atas'
        hitung_benar(ujian_siswas_atas, keterangan)
        
        #nilai_rendah
        ujian_siswas_bawah = get_ujian_siswa_kategori('ASC', paket['id'], batas_jumlah)
        keterangan = 'benar_bawah'
        hitung_benar(ujian_siswas_bawah, keterangan)

        for index, soal in soal.iterrows():
            try:
                daya_pembeda = (soal['benar_atas']-soal['benar_bawah'])/(batas_jumlah)
                print(daya_pembeda)
            except:
                daya_pembeda = -2
                print("soal: ", index)
            sql = "update soal_analisis set daya_pembeda=%f where id='%s'" % (daya_pembeda, index)
            cur_thread.execute(sql)
    sql = "update paket set status=5 where id='%s'" % (paket['id'])
    cur_thread.execute(sql)
    db_thread.commit()
    
    print("~DONE~ paket:", paket['nama'])

soal:  1f8f239c-5783-11ea-b304-86cbb67d2138
soal:  1f8f23a1-5783-11ea-b304-86cbb67d2138
soal:  1f8f23a6-5783-11ea-b304-86cbb67d2138
soal:  1f8f23ab-5783-11ea-b304-86cbb67d2138
soal:  1f8f23b0-5783-11ea-b304-86cbb67d2138
soal:  1f8f23b5-5783-11ea-b304-86cbb67d2138
soal:  1f8f23ba-5783-11ea-b304-86cbb67d2138
soal:  1f8f23c0-5783-11ea-b304-86cbb67d2138
soal:  1f8f23c4-5783-11ea-b304-86cbb67d2138
soal:  1f8f23c9-5783-11ea-b304-86cbb67d2138
soal:  1f8f23ce-5783-11ea-b304-86cbb67d2138
soal:  1f8f23d3-5783-11ea-b304-86cbb67d2138
soal:  1f8f23d8-5783-11ea-b304-86cbb67d2138
soal:  1f8f23dd-5783-11ea-b304-86cbb67d2138
soal:  1f8f23e2-5783-11ea-b304-86cbb67d2138
soal:  1f8f23e7-5783-11ea-b304-86cbb67d2138
soal:  1f8f23ec-5783-11ea-b304-86cbb67d2138
soal:  1f8f23f1-5783-11ea-b304-86cbb67d2138
soal:  1f8f23f6-5783-11ea-b304-86cbb67d2138
soal:  1f8f23fc-5783-11ea-b304-86cbb67d2138
soal:  1f8f2400-5783-11ea-b304-86cbb67d2138
soal:  1f8f2405-5783-11ea-b304-86cbb67d2138
soal:  1f8f240a-5783-11ea-b304-8

0.15486725663716813
0.008849557522123894
0.40486725663716816
0.14380530973451328
0.1261061946902655
0.022123893805309734
0.12831858407079647
0.6194690265486725
0.584070796460177
0.13938053097345132
0.0
0.22566371681415928
0.008849557522123894
0.3694690265486726
0.27654867256637167
0.2411504424778761
0.504424778761062
0.6924778761061947
0.24557522123893805
0.6570796460176991
0.024336283185840708
0.13938053097345132
0.5243362831858407
0.5951327433628318
0.4358407079646018
0.2013274336283186
0.18141592920353983
0.0
0.32079646017699115
0.5420353982300885
0.0
0.2743362831858407
0.13495575221238937
0.3938053097345133
0.008849557522123894
0.0022123893805309734
0.36283185840707965
0.4823008849557522
0.5508849557522124
0.4646017699115044
0.0
0.0
0.1261061946902655
0.16371681415929204
0.0420353982300885
~DONE~ paket: KTSP_Teknologi_Informasi_dan_Komunikasi_2
0.1782608695652174
0.11304347826086956
0.4782608695652174
0.10869565217391304
0.1
0.08695652173913043
0.22608695652173913
0.356521739130434

0.6046511627906976
0.5348837209302325
0.6744186046511628
0.6976744186046512
0.0
0.6744186046511628
0.3953488372093023
0.5348837209302325
0.6046511627906976
-0.13953488372093023
0.5581395348837209
0.6511627906976745
0.5348837209302325
0.6511627906976745
0.4883720930232558
0.7441860465116279
0.6046511627906976
0.6046511627906976
0.4186046511627907
0.6046511627906976
0.0
-0.13953488372093023
0.4883720930232558
0.6511627906976745
0.46511627906976744
0.3953488372093023
0.09302325581395349
0.6976744186046512
0.5116279069767442
0.5116279069767442
0.023255813953488372
0.5348837209302325
0.0
-0.06976744186046512
0.5813953488372093
0.5116279069767442
0.6744186046511628
0.5348837209302325
0.7209302325581395
0.0
0.3488372093023256
0.5581395348837209
0.32558139534883723
-0.046511627906976744
0.0
~DONE~ paket: KTSP_Bahasa_Mandarin_3
0.07142857142857142
0.2857142857142857
0.21428571428571427
0.0
0.21428571428571427
0.21428571428571427
0.35714285714285715
0.42857142857142855
0.2857142857142857
-0.1428

0.32642487046632124
0.32642487046632124
0.3471502590673575
0.19170984455958548
0.26424870466321243
0.37305699481865284
0.21761658031088082
0.22797927461139897
0.36787564766839376
0.046632124352331605
0.15544041450777202
0.0
0.20207253886010362
0.23834196891191708
0.13471502590673576
0.0
0.12953367875647667
0.38341968911917096
0.20725388601036268
0.48186528497409326
0.13471502590673576
0.22797927461139897
0.09326424870466321
0.0
0.2694300518134715
0.19170984455958548
0.41968911917098445
0.10362694300518134
0.33678756476683935
0.20207253886010362
0.0
0.16062176165803108
0.09844559585492228
0.21243523316062177
0.0
0.20207253886010362
0.08290155440414508
0.04145077720207254
0.30569948186528495
0.27461139896373055
~DONE~ paket: KTSP_Fisika_1
0.2837209302325581
0.2930232558139535
0.30697674418604654
0.07906976744186046
0.3116279069767442
0.3581395348837209
0.16744186046511628
0.22325581395348837
0.2558139534883721
0.24186046511627907
0.26046511627906976
0.0
0.2837209302325581
0.2604651162790

0.16730038022813687
0.43346007604562736
0.17110266159695817
0.3574144486692015
0.23574144486692014
0.1482889733840304
0.5057034220532319
0.060836501901140684
0.0
0.0
0.1482889733840304
0.1596958174904943
0.311787072243346
0.13307984790874525
0.19011406844106463
0.3041825095057034
0.0
0.25475285171102663
0.18631178707224336
0.26996197718631176
0.04182509505703422
0.4790874524714829
0.29277566539923955
0.1444866920152091
0.3231939163498099
0.10646387832699619
0.18250950570342206
0.0
0.10646387832699619
0.19391634980988592
0.12167300380228137
0.09505703422053231
0.08365019011406843
0.2737642585551331
0.4524714828897338
0.30798479087452474
0.23954372623574144
0.0
0.1482889733840304
0.22813688212927757
~DONE~ paket: KTSP_Ekonomi_2
0.25609756097560976
0.47560975609756095
0.36585365853658536
0.40853658536585363
0.15853658536585366
0.12195121951219512
0.10365853658536585
0.11585365853658537
0.0
0.3597560975609756
0.12195121951219512
0.15853658536585366
0.2073170731707317
0.17682926829268292
0.

0.13876343248715153
0.4617660800498365
0.503815605045943
0.3516586201526242
0.4955614390281888
0.35819965737424075
0.458028344494627
0.39573275190780255
0.5932097804080362
0.5569225977262109
0.6276280953122566
0.48076623578881794
0.0
0.5209468930073198
0.5209468930073198
0.6558168509577947
0.5284223641177387
0.3533717489487619
0.0
0.4295281108861548
0.5536520791154026
0.39074910450085654
0.0
0.6171935835539636
0.33857654570939105
0.5058402118050148
0.501168042361003
0.7428749415978819
0.5380781809686964
0.165394798318019
0.38810154181591655
0.7391372060426725
0.5899392617972279
0.41940507709079583
0.0
0.4905777916212428
0.19809998442610185
0.6182837564242329
0.38560971811244354
0.5835539635570783
0.5664226755957016
0.239215075533406
0.5851113533717489
0.0
0.5657997196698333
~DONE~ paket: K13_Pendidikan_Pancasila_dan_Kewarganegaraan_1
0.5630812151270924
0.5669559826410415
0.05176689398636082
0.6881587104773713
0.2490700557966522
0.42994420334779915
0.24008059516429014
0.1627402355858648

0.41254125412541254
0.46864686468646866
0.5445544554455446
0.6171617161716172
0.1617161716171617
0.636963696369637
0.23102310231023102
0.1551155115511551
0.19801980198019803
0.3333333333333333
0.28052805280528054
0.0
0.3465346534653465
0.44224422442244227
0.4752475247524752
0.30363036303630364
0.49504950495049505
0.43234323432343236
0.0165016501650165
0.0
0.43564356435643564
0.38943894389438943
0.20462046204620463
0.5643564356435643
0.3795379537953795
0.48184818481848185
0.0
0.15841584158415842
0.31353135313531355
0.33003300330033003
0.5973597359735974
0.45874587458745875
0.264026402640264
0.5511551155115512
0.07260726072607261
0.30363036303630364
0.0
0.41254125412541254
0.41254125412541254
0.0
~DONE~ paket: K13_Bahasa_dan_Sastra_Indonesia_1
0.405982905982906
0.5811965811965812
0.594017094017094
0.44871794871794873
0.24358974358974358
0.5042735042735043
0.3076923076923077
0.4188034188034188
0.5811965811965812
0.3247863247863248
0.3803418803418803
0.0
0.5213675213675214
0.51282051282051